<h1>Snow Violations, 2019-2023</h1>
11 January 2024

START BY READING MY NEW FILE

This analysis looks at 311 complaints filed as "Snow – Uncleared Sidewalk Complaint," or SWSNOREM.<br>
<br>
My analysis steps:
<ol>
<li><a href="#read">Read Data</a>
<li><a href="#ward">By Ward, 1-Year and 5-Year</a>
<li>Ward X
</ol>

<div style="color:red;">
    <h3>Preliminary Findings</h3>
<ul>
    <li>...
</ul>
    </div>

<a name="read"></a>
# 1. Read Data

In [1]:
import pandas as pd
import requests
#import datetime as dt #would only need this if I manipulated dates post-API data retrieval

In [22]:
df = pd.read_csv("../data/311-snow-violations.csv")
df.head()

SR_NUMBER SR_SHORT_CODE         CREATED_DATE       STREET_ADDRESS  \
0  SR21-00149474           SDO  2021-01-27 13:13:50       1700 W 15TH ST   
1  SR21-00177207           SDO  2021-02-01 10:19:34     1300 S HEATH AVE   
2  SR21-00269268           SDO  2021-02-17 12:43:47      819 S BISHOP ST   
3  SR21-00149811           SDO  2021-01-27 13:50:04   1300 S WOLCOTT AVE   
4  SR21-00188251           SDO  2021-02-02 15:47:03  1200 W LEXINGTON ST   

   COMMUNITY_AREA  WARD     STATUS         ORIGIN          CLOSED_DATE  \
0              28    28  Completed  Mobile Device  2021-01-27 19:50:47   
1              28    28  Completed  Mobile Device  2021-02-01 15:12:41   
2              28    28  Completed  Mobile Device  2021-02-19 00:58:56   
3              28    28  Completed  Mobile Device  2021-01-27 19:10:36   
4              28    28  Completed  Mobile Device  2021-02-02 21:53:42   

    LATITUDE  LONGITUDE                       SR_TYPE  year  month  \
0  41.861457 -87.668881  Ice and Snow Removal Request  2021      1   
1  41.864744 -87.684402  Ice and Snow Removal Request  2021      2   
2  41.871081 -87.662624  Ice and Snow Removal Request  2021      2   
3  41.864893 -87.673790  Ice and Snow Removal Request  2021      1   
4  41.872287 -87.656946  Ice and Snow Removal Request  2021      2   

         date     season  GEOID  COMMUNITY_NAME  
0  2021-01-27  2020-2021     28  Near West Side  
1  2021-02-01  2020-2021     28  Near West Side  
2  2021-02-17  2020-2021     28  Near West Side  
3  2021-01-27  2020-2021     28  Near West Side  
4  2021-02-02  2020-2021     28  Near West Side

In [23]:
len(df)

68273

### filter dataset
for the following:
<ul>
    <li>just full seasons (July 2019 to June 2023)
        <li>just uncleared sidewalk complaints
    <li>just one community (Lower West Side, for now)        
    </ul>

In [36]:
df = df[(df['season'].isin(['2019-2020','2020-2021','2021-2022','2022-2023'])) &
       (df['SR_SHORT_CODE']=='SWSNOREM') & \
        (df['COMMUNITY_NAME']=='Lower West Side')
       ]
len(df)

325

# Repeat Offenders

In [60]:
df_by_address = df.groupby('STREET_ADDRESS').agg({
                   'LATITUDE': 'first', 
                   'LONGITUDE': 'first',
                   'CREATED_DATE': ['min', 'max'],
                   'STREET_ADDRESS': 'size'
                })

# Flatten the multi-level columns
df_by_address.columns = ['_'.join(col) if col[1] else col[0] for col in df_by_address.columns.values]

# Rename columns
df_by_address = df_by_address.rename(columns={
                    'CREATED_DATE_min': 'first_complaint_date', 
                    'CREATED_DATE_max': 'last_complaint_date',
                    'STREET_ADDRESS_size': 'Complaints'
                })

# Sort and reset index
df_by_address = df_by_address.sort_values('Complaints', ascending=False).reset_index()

df_by_address.head()

STREET_ADDRESS  LATITUDE_first  LONGITUDE_first first_complaint_date  \
0       1339 W 19TH ST       41.855911       -87.660365  2019-11-11 22:06:00   
1      813 W CERMAK RD       41.852355       -87.647319  2020-01-20 10:10:09   
2        947 W 18TH ST       41.857872       -87.650560  2021-02-03 18:09:16   
3       1800 W 17TH ST       41.858906       -87.671302  2022-01-24 11:55:48   
4  1810 W CULLERTON ST       41.855177       -87.671549  2022-01-04 19:04:44   

   last_complaint_date  Complaints  
0  2022-01-15 16:44:20           6  
1  2020-02-26 15:39:08           6  
2  2022-02-03 07:37:44           5  
3  2022-02-27 11:05:24           5  
4  2023-02-01 22:05:26           5

In [61]:
df_repeat = df_by_address[df_by_address['Complaints']>1]
df_repeat.head()

STREET_ADDRESS  LATITUDE_first  LONGITUDE_first first_complaint_date  \
0       1339 W 19TH ST       41.855911       -87.660365  2019-11-11 22:06:00   
1      813 W CERMAK RD       41.852355       -87.647319  2020-01-20 10:10:09   
2        947 W 18TH ST       41.857872       -87.650560  2021-02-03 18:09:16   
3       1800 W 17TH ST       41.858906       -87.671302  2022-01-24 11:55:48   
4  1810 W CULLERTON ST       41.855177       -87.671549  2022-01-04 19:04:44   

   last_complaint_date  Complaints  
0  2022-01-15 16:44:20           6  
1  2020-02-26 15:39:08           6  
2  2022-02-03 07:37:44           5  
3  2022-02-27 11:05:24           5  
4  2023-02-01 22:05:26           5

In [62]:
len(df_repeat)

53

In [64]:
df_repeat.to_csv("../results/repeat-complaints-lower-west-side.csv", index=False)

### Read assessor parcel info

In [65]:
df_assessor=pd.read_csv("../data/Assessor_-_Parcel_Universe_20240111.csv")

In [69]:
df_assessor = df_assessor[[,,]]

<bound method DataFrame.info of                   pin       pin10  tax_year class triad_name  triad_code  \
0      17213130370000  1721313037      2019    EX       City           1   
1      17213010040000  1721301004      2019   100       City           1   
2      17204270210000  1720427021      2018   212       City           1   
3      17301150150000  1730115015      2018   211       City           1   
4      17301200210000  1730120021      2023   203       City           1   
...               ...         ...       ...   ...        ...         ...   
45835  17193000390000  1719300039      2021   590       City           1   
45836  17301070430000  1730107043      2019   211       City           1   
45837  17301090300000  1730109030      2019   211       City           1   
45838  17213030080000  1721303008      2019    EX       City           1   
45839  17301210381018  1730121038      2018   299       City           1   

      township_name  township_code  neighborhood_code  